In [10]:
from Twitter.analysis_v2.data_analysis import *
from os import walk

# Data Import and overview

In [11]:
DATASETS_PATH = "../../data/processed_tweets/"

processed_filenames = next(walk(DATASETS_PATH), (None, None, []))[2]
filenames = [DATASETS_PATH + "/" + filename for filename in processed_filenames]

In [12]:
df = ensemble_dataset(filenames)

In [13]:
df['year'].value_counts()

2021    533375
2019    531272
2020    531227
Name: year, dtype: int64

## Tweets performance

In [14]:
topics_categories = df['topics_cleaned'].unique()[1:]
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
week_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_phases = ['Morning', 'Afternoon', 'Dusk', 'Night', 'Middle of the night']
day_phases_old = ['Dawn', 'Morning', 'Afternoon', 'Evening', 'Night']
sentiments = ['Negative', 'Neutral', 'Positive']
hashtags = [True, False]
offline_graphs = False

In [15]:
tweet_analysis = df[['text', 'year', 'day_phase', 'day_of_week', 'month', 'retweet_count', 'quote_count', 'like_count', 'reply_count', 'sentiment', 'hashtags', 'topics_cleaned']]

### Average retweet and like count per phase of the day

In [16]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['day_phase'], day_phases)
df_analysis

,year,day_phase,count day_phase,% day_phase,% with retweets,% with likes,retweets mean,likes mean
0,2019,Morning,105912,19.935551,14.37,44.76,0.41,2.45
1,2019,Afternoon,106273,20.003501,17.78,48.18,0.56,3.40
2,2019,Dusk,106415,20.030229,18.12,50.00,0.56,3.62
3,2019,Night,106312,20.010842,17.73,49.82,0.54,3.64
4,2019,Middle of the night,106360,20.019877,17.32,50.74,0.52,3.68
0,2020,Morning,105633,19.884720,13.95,45.36,0.41,2.63
1,2020,Afternoon,106420,20.032867,16.94,47.80,0.54,3.67
2,2020,Dusk,106397,20.028538,16.95,49.96,0.53,3.84
3,2020,Night,106408,20.030608,16.73,50.41,0.51,3.91
4,2020,Middle of the night,106369,20.023267,16.44,51.16,0.48,3.88


In [17]:
analysis_chart(df_analysis, 'day_phase', '% with retweets', '% with likes', 'Day phase', '% with retweets', '% with likes', 'Percentage of retweets and likes during the day', offline_graphs)
analysis_chart(df_analysis, 'day_phase', 'retweets mean', 'likes mean', 'Day phase', 'Retweets mean', 'Likes mean', 'Average retweets and likes during the day', offline_graphs)

In [44]:
analysis_chart(df_analysis, 'day_of_week', '% with retweets', '% with likes', 'Weekday', '% with retweets',
               '% with likes', 'Percentage of retweets and likes during the week', offline_graphs)
analysis_chart(df_analysis, 'day_of_week', 'retweets mean', 'likes mean', 'Weekday', 'Retweets mean', 'Likes mean',
               'Average retweets and likes during the week', offline_graphs)

### Average retweet count per month

In [45]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['month'], months)
df_analysis

,year,month,count month,% month,% with retweets,% with likes,retweets mean,likes mean
0,2019,January,45257,8.518612,16.24,46.04,0.49,2.90
1,2019,February,40908,7.700011,16.79,48.17,0.48,2.98
2,2019,March,45287,8.524259,16.83,47.29,0.51,3.30
3,2019,April,43801,8.244553,17.28,48.78,0.54,3.38
4,2019,May,44208,8.321161,17.23,48.57,0.53,3.41
5,2019,June,43778,8.240223,16.89,48.14,0.50,3.19
6,2019,July,45211,8.509953,16.27,47.02,0.47,3.09
7,2019,August,45179,8.503930,17.09,49.70,0.51,3.43
8,2019,September,43741,8.233259,17.53,49.90,0.54,3.53
9,2019,October,45060,8.481531,17.69,50.22,0.55,3.60


In [46]:
analysis_chart(df_analysis, 'month', '% with retweets', '% with likes', 'Month', '% with retweets',
               '% with likes', 'Percentage of retweets and likes during the year', offline_graphs)
analysis_chart(df_analysis, 'month', 'retweets mean', 'likes mean', 'Month', 'Retweets mean', 'Likes mean',
               'Average retweets and likes during the year', offline_graphs)

### Tweets performance by sentiment

In [47]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['sentiment'], sentiments)
df_analysis

,year,sentiment,count sentiment,% sentiment,% with retweets,% with likes,retweets mean,likes mean
0,2019,Negative,125989,23.714594,16.18,46.80,0.52,2.90
1,2019,Neutral,163721,30.816794,15.18,44.15,0.42,2.80
2,2019,Positive,241562,45.468611,18.80,52.78,0.59,3.98
0,2020,Negative,131147,24.687563,15.74,48.13,0.50,3.15
1,2020,Neutral,165832,31.216787,14.23,43.83,0.40,2.98
2,2020,Positive,234248,44.095650,17.86,53.02,0.55,4.26
0,2021,Negative,121368,22.754722,15.23,48.73,0.50,3.83
1,2021,Neutral,181050,33.944223,13.51,43.54,0.40,3.73
2,2021,Positive,230957,43.301055,18.52,55.59,0.60,5.48


In [48]:
analysis_chart(df_analysis, 'sentiment', '% with retweets', '% with likes', 'Sentiment', 'Retweet count',
               'Likes count', 'Percentage of retweets and likes by sentiment', offline_graphs)
analysis_chart(df_analysis, 'sentiment', 'retweets mean', 'likes mean', 'Sentiment', 'Retweets mean', 'Likes mean',
               'Average retweets and likes number by sentiment', offline_graphs)

### Tweets performance by topics

#### Performance of each topic in retweets and likes

In [49]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['topics_cleaned'], topics_categories)
df_analysis

,year,topics_cleaned,count topics_cleaned,% topics_cleaned,% with retweets,% with likes,retweets mean,likes mean
0,2019,Other,32323.0,16.662457,16.14,46.38,0.52,3.30
1,2019,Person,36681.0,18.908999,15.45,44.01,0.58,3.30
2,2019,TV and Movies,34474.0,17.771294,15.33,51.57,0.47,3.91
3,2019,Entities,20770.0,10.706903,15.13,50.12,0.43,3.21
4,2019,Interest and Hobbies,26951.0,13.893199,15.07,48.49,0.42,3.15
5,2019,Sport,7326.0,3.776542,22.17,59.10,0.66,5.54
6,2019,Brand,29804.0,15.363916,16.23,48.83,0.50,3.17
7,2019,Music,2036.0,1.049555,19.50,52.90,0.47,2.94
8,2019,Video Game,1985.0,1.023264,16.07,55.01,0.40,3.83
9,2019,Political,1428.0,0.736132,14.99,31.86,1.05,2.02


In [50]:
analysis_chart(df_analysis, 'topics_cleaned', '% with retweets', '% with likes', 'Topics', 'Retweet count',
               'Likes count', 'Percentage of retweets and likes by topic', offline_graphs)
analysis_chart(df_analysis, 'topics_cleaned', 'retweets mean', 'likes mean', 'Topics', 'Retweets mean', 'Likes mean',
               'Average tweets performance by topic', offline_graphs)

#### Average retweet count per topic during the day

In [51]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['day_phase', 'topics_cleaned'], day_phases)
df_analysis.head(60)

,year,day_phase,topics_cleaned,count day_phase,% day_phase,% with retweets,% with likes,retweets mean,likes mean
0,2019,Afternoon,Book,32,0.016496,37.50,59.38,1.12,8.09
1,2019,Afternoon,Brand,6028,3.107425,17.47,48.59,0.49,3.13
2,2019,Afternoon,Entities,4331,2.232624,16.37,50.13,0.50,3.28
3,2019,Afternoon,Interest and Hobbies,5915,3.049173,16.04,48.45,0.49,3.34
4,2019,Afternoon,Music,383,0.197436,21.41,48.56,0.48,2.92
5,2019,Afternoon,Other,7761,4.000784,17.60,46.40,0.51,3.27
6,2019,Afternoon,Person,8588,4.427101,15.80,42.23,0.55,3.27
7,2019,Afternoon,Political,395,0.203622,14.18,30.89,1.06,2.77
8,2019,Afternoon,Sport,1335,0.688190,26.59,62.32,0.85,7.02
9,2019,Afternoon,TV and Movies,5238,2.700181,18.00,53.53,0.60,4.38


In [52]:
multi_label_chart(df_analysis, "topics_cleaned", day_phases, "day_phase", "% with retweets", "Day phase", "% with retweets", "Percentage of retweets by topic during the day", offline_graphs)

#### Average retweet count per topic during the week

In [53]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['day_of_week', 'topics_cleaned'], week_days)
df_analysis

,year,day_of_week,topics_cleaned,count day_of_week,% day_of_week,% with retweets,% with likes,retweets mean,likes mean
0,2019,Friday,Book,30,0.015465,20.00,56.67,0.20,2.77
1,2019,Friday,Brand,4227,2.179012,16.63,48.78,0.50,3.23
2,2019,Friday,Entities,2987,1.539794,15.03,49.15,0.40,3.21
3,2019,Friday,Interest and Hobbies,3863,1.991371,13.72,47.04,0.40,2.96
4,2019,Friday,Music,347,0.178878,15.56,49.28,0.26,1.88
...,...,...,...,...,...,...,...,...,...
86,2021,Wednesday,Person,4851,2.786202,16.47,48.34,0.62,4.32
87,2021,Wednesday,Political,243,0.139569,20.99,44.86,1.40,4.97
88,2021,Wednesday,Sport,1083,0.622028,25.58,63.80,0.79,7.53
89,2021,Wednesday,TV and Movies,5374,3.086590,15.41,54.82,0.52,5.67


In [54]:
multi_label_chart(df_analysis, "topics_cleaned", week_days, "day_of_week", "% with retweets", "Weekday", "% with retweets", "Percentage of retweets by topic during the week", offline_graphs)

#### Average retweet count per topic during the year

In [55]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['month', 'topics_cleaned'], months)
df_analysis

,year,month,topics_cleaned,count month,% month,% with retweets,% with likes,retweets mean,likes mean
0,2019,April,Book,24,0.012372,37.50,58.33,1.96,8.12
1,2019,April,Brand,2303,1.187193,15.59,47.37,0.41,3.08
2,2019,April,Entities,1490,0.768093,15.64,49.93,0.36,2.85
3,2019,April,Interest and Hobbies,2183,1.125333,16.31,47.82,0.42,3.06
4,2019,April,Music,174,0.089697,24.14,55.75,0.53,3.22
...,...,...,...,...,...,...,...,...,...
151,2021,September,Person,2535,1.455993,17.16,49.31,0.58,4.36
152,2021,September,Political,118,0.067774,22.03,42.37,0.82,2.72
153,2021,September,Sport,545,0.313024,24.22,59.27,0.57,5.83
154,2021,September,TV and Movies,4022,2.310060,15.61,54.18,0.49,5.74


In [56]:
multi_label_chart(df_analysis, "topics_cleaned", months, "month", "% with retweets", "Months", "% with retweets", "Percentage of retweets by topic during the year", offline_graphs)

#### Impact of hashtags in topic popularity

In [57]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['hashtags', 'topics_cleaned'], hashtags)
df_analysis

,year,hashtags,topics_cleaned,count hashtags,% hashtags,% with retweets,% with likes,retweets mean,likes mean
0,2019,False,Book,131,0.067530,24.43,58.78,0.68,6.40
1,2019,False,Brand,23448,12.087408,15.09,49.17,0.46,3.09
2,2019,False,Entities,15986,8.240758,14.14,50.78,0.39,3.14
3,2019,False,Interest and Hobbies,18669,9.623841,14.95,52.38,0.43,3.42
4,2019,False,Music,1394,0.718605,19.15,54.23,0.50,3.49
...,...,...,...,...,...,...,...,...,...
21,2021,True,Person,4843,2.781607,24.45,52.84,0.98,5.23
22,2021,True,Political,250,0.143589,29.20,46.80,1.22,4.33
23,2021,True,Sport,2896,1.663335,26.66,63.36,0.92,8.05
24,2021,True,TV and Movies,14448,8.298298,18.77,58.02,0.59,6.29


In [58]:
multi_label_chart(df_analysis, "topics_cleaned", topics_categories, "hashtags", "% with retweets", "hashtags", "% with retweets", "Tweet sentiment by topic and corresponding % retweet count", offline_graphs)

#### Tweet sentiment per topic

In [59]:
df_analysis = retweets_likes_info_by_year(tweet_analysis, ['sentiment', 'topics_cleaned'], sentiments)
df_analysis

,year,sentiment,topics_cleaned,count sentiment,% sentiment,% with retweets,% with likes,retweets mean,likes mean
0,2019,Negative,Book,44,0.022682,15.91,47.73,0.25,1.86
1,2019,Negative,Brand,6595,3.399712,15.71,49.78,0.64,3.09
2,2019,Negative,Entities,4063,2.094470,15.46,50.11,0.58,3.62
3,2019,Negative,Interest and Hobbies,5987,3.086289,14.26,44.95,0.42,2.84
4,2019,Negative,Music,388,0.200013,19.07,49.74,0.52,3.17
...,...,...,...,...,...,...,...,...,...
34,2021,Positive,Person,13507,7.757828,17.54,52.06,0.63,5.02
35,2021,Positive,Political,564,0.323937,23.94,51.06,1.41,6.21
36,2021,Positive,Sport,4250,2.441014,27.13,69.20,0.80,8.21
37,2021,Positive,TV and Movies,18928,10.871413,17.23,58.05,0.52,6.30


In [60]:
multi_label_chart(df_analysis, "topics_cleaned", topics_categories, "sentiment", "% with retweets", "sentiment", "% with retweets", "Tweet sentiment by topic and corresponding % retweet count", offline_graphs)